# Load dataset & problem

In [1]:
import pickle

from d3m_ta2_nyu.d3mds import D3MDS

In [2]:
ds = D3MDS('data/LL0_21_car/LL0_21_car_dataset',
           'data/LL0_21_car/LL0_21_car_problem')

In [3]:
train_data = ds.get_train_data()
train_data.head()

,buying,maint,doors,persons,lug_boot,safety
d3mIndex,,,,,,
1,vhigh,vhigh,2,2,small,med
2,vhigh,vhigh,2,2,small,high
3,vhigh,vhigh,2,2,med,low
5,vhigh,vhigh,2,2,med,high
6,vhigh,vhigh,2,2,big,low


In [4]:
train_targets = ds.get_train_targets()

In [5]:
test_data = ds.get_test_data()
test_data.head()

,buying,maint,doors,persons,lug_boot,safety
d3mIndex,,,,,,
0,vhigh,vhigh,2,2,small,low
4,vhigh,vhigh,2,2,med,med
9,vhigh,vhigh,2,4,small,low
11,vhigh,vhigh,2,4,small,high
22,vhigh,vhigh,2,more,med,med


In [6]:
test_targets = ds.get_test_targets()

In [7]:
len(train_data), len(train_targets), len(test_data), len(test_targets)

(1383, 1383, 345, 345)

# Imputation using dsbox

In [8]:
from dsbox.datapreprocessing.cleaner import KNNImputation, KnnHyperparameter

Using TensorFlow backend.
/Users/remram/Documents/programming/_venvs/d3m/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [9]:
imputer = KNNImputation(hyperparams=KnnHyperparameter.defaults())

In [10]:
results = imputer.produce(inputs=train_data)
assert results.has_finished
train_data = results.value
train_data.head()

Exception in thread Thread-4:
Traceback (most recent call last):
  File "/opt/local/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/opt/local/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/threading.py", line 1180, in run
    self.finished.wait(self.interval)
  File "/opt/local/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/threading.py", line 551, in wait
    signaled = self._cond.wait(timeout)
  File "/opt/local/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/threading.py", line 299, in wait
    gotit = waiter.acquire(True, timeout)
OverflowError: timestamp too large to convert to C _PyTime_t



,buying,maint,doors,persons,lug_boot,safety
d3mIndex,,,,,,
1,vhigh,vhigh,2,2,small,med
2,vhigh,vhigh,2,2,small,high
3,vhigh,vhigh,2,2,med,low
5,vhigh,vhigh,2,2,med,high
6,vhigh,vhigh,2,2,big,low


In [11]:
results = imputer.produce(inputs=test_data)
assert results.has_finished
test_data = results.value
test_data.head()

Exception in thread Thread-5:
Traceback (most recent call last):
  File "/opt/local/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/opt/local/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/threading.py", line 1180, in run
    self.finished.wait(self.interval)
  File "/opt/local/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/threading.py", line 551, in wait
    signaled = self._cond.wait(timeout)
  File "/opt/local/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/threading.py", line 299, in wait
    gotit = waiter.acquire(True, timeout)
OverflowError: timestamp too large to convert to C _PyTime_t



,buying,maint,doors,persons,lug_boot,safety
d3mIndex,,,,,,
0,vhigh,vhigh,2,2,small,low
4,vhigh,vhigh,2,2,med,med
9,vhigh,vhigh,2,4,small,low
11,vhigh,vhigh,2,4,small,high
22,vhigh,vhigh,2,more,med,med


# Encoding using dsbox

In [12]:
from dsbox.datapreprocessing.cleaner import Encoder, EncHyperparameter

In [13]:
encoder = Encoder(hyperparams=EncHyperparameter.defaults())

In [15]:
encoder.set_training_data(inputs=train_data)
encoder.fit()

In [15]:
params = pickle.loads(pickle.dumps(encoder.get_params()))

In [16]:
hparams = EncHyperparameter(pickle.loads(pickle.dumps(dict(encoder.hyperparams))))

In [17]:
encoder = Encoder(hyperparams=hparams)
encoder.set_params(params=params)

In [18]:
results = encoder.produce(inputs=train_data)
assert results.has_finished
train_data = results.value
train_data.head()

,buying_high,buying_low,buying_med,buying_vhigh,buying_nan,buying_other_,maint_high,maint_low,maint_med,maint_vhigh,...,lug_boot_big,lug_boot_med,lug_boot_small,lug_boot_nan,lug_boot_other_,safety_high,safety_low,safety_med,safety_nan,safety_other_
d3mIndex,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,1,0,0,0,0,0,1,...,0,0,1,0,0,0,0,1,0,0
2,0,0,0,1,0,0,0,0,0,1,...,0,0,1,0,0,1,0,0,0,0
3,0,0,0,1,0,0,0,0,0,1,...,0,1,0,0,0,0,1,0,0,0
5,0,0,0,1,0,0,0,0,0,1,...,0,1,0,0,0,1,0,0,0,0
6,0,0,0,1,0,0,0,0,0,1,...,1,0,0,0,0,0,1,0,0,0


In [19]:
results = encoder.produce(inputs=test_data)
assert results.has_finished
test_data = results.value
test_data.head()

,buying_high,buying_low,buying_med,buying_vhigh,buying_nan,buying_other_,maint_high,maint_low,maint_med,maint_vhigh,...,lug_boot_big,lug_boot_med,lug_boot_small,lug_boot_nan,lug_boot_other_,safety_high,safety_low,safety_med,safety_nan,safety_other_
d3mIndex,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,1,0,0,0,0,0,1,...,0,0,1,0,0,0,1,0,0,0
4,0,0,0,1,0,0,0,0,0,1,...,0,1,0,0,0,0,0,1,0,0
9,0,0,0,1,0,0,0,0,0,1,...,0,0,1,0,0,0,1,0,0,0
11,0,0,0,1,0,0,0,0,0,1,...,0,0,1,0,0,1,0,0,0,0
22,0,0,0,1,0,0,0,0,0,1,...,0,1,0,0,0,0,0,1,0,0


# Classification

In [20]:
ds.problem.prDoc['about']['taskType'], ds.problem.prDoc['about']['taskSubType']

('classification', 'multiClass')

In [21]:
from sklearn.neighbors.classification import KNeighborsClassifier

In [22]:
classifier = KNeighborsClassifier()

In [23]:
classifier.fit(train_data, train_targets)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [24]:
prediction = classifier.predict(test_data)

In [25]:
print("{} / {}".format(sum(1 for i, j in zip(prediction, test_targets) if i == j), len(test_targets)))

295 / 345
